## Explore and clean data

In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import statistics
%matplotlib inline

In [36]:
gltbc = pd.read_csv("./data/climate-change-earth-surface-temperature-data/GlobalLandTemperaturesByCountry.csv")
gltbc.head(5)

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


In [37]:
#Dropping all rows with NaN in AverageTemperature column
gltbc_clean = gltbc[gltbc['AverageTemperature'].isnull() == False].copy()

In [38]:
#Cast 'dt' colum to DateTime object
gltbc_clean['dt'] = gltbc_clean['dt'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

In [39]:
#Select Germany subset
gltbc_ger = gltbc_clean[gltbc['Country'] == 'Germany'].copy()

C:\Users\wacke\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [40]:
#Add column with saisons
def season(date):
    if date.month < 3 or date.month == 12:
        return "Winter"
    
    elif date.month > 2 and date.month < 6:
        return "Spring"
    
    elif date.month > 5 and date.month < 9:
        return "Summer"
    
    else:
        return 'Autumn'
    
gltbc_ger['Season'] = gltbc_ger['dt'].apply(season)

## Descriptive Statistics

1a) Highest temperature, lowest, mean, median and mad

In [64]:
#Calculate the max, min, mean and median
max_temp = gltbc_ger['AverageTemperature'].max()
min_temp = gltbc_ger['AverageTemperature'].min()
mean_temp = gltbc_ger['AverageTemperature'].mean()
median_temp = gltbc_ger['AverageTemperature'].median()
mad_temp = gltbc_ger['AverageTemperature'].mad()

#Create DataFrame with values
data = {'Temperature in °C':map(lambda x: round(x, 2), [max_temp, min_temp, mean_temp, median_temp, mad_temp])}
stats = pd.DataFrame(data, index = ['max_temp min_temp mean_temp median_temp mad_temp'.split(" ")])
stats

,Temperature in °C
max_temp,22.34
min_temp,-9.30
mean_temp,8.15
median_temp,8.04
mad_temp,6.08
